# SI 330 Class 7: More data manipulation with pandas 


In [1]:
# load a few libraries we'll need

%matplotlib inline
import pandas as pd
import random
import numpy as np
from random import sample

## Some more basic pandas functionality
One of the nice things about pandas is that it simplifies many common operations on datasets.  Let's load the LOTR dataset
from last class:



## Pivots/Stack/Unstack Intro
The following cells are based on: 
http://nikgrozev.com/2015/07/01/reshaping-in-pandas-pivot-pivot-table-stack-and-unstack-explained-with-pictures/

In [2]:
from collections import OrderedDict
from pandas import DataFrame

table = OrderedDict((
    ("Item", ['Item0', 'Item0', 'Item1', 'Item1']),
    ('CType',['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD',  ['1$', '2$', '3$', '4$']),
    ('EU',   ['1€', '2€', '3€', '4€'])
))
metal = DataFrame(table)
metal

# create the table

,Item,CType,USD,EU
0,Item0,Gold,1$,1€
1,Item0,Bronze,2$,2€
2,Item1,Gold,3$,3€
3,Item1,Silver,4$,4€


In [3]:
# make a table of items (rows) and costs (USD) 
# for each in gold and bronze
metal.pivot(index='Item',columns='CType',values='USD')

# see the image below... we're telling Pandas to take the table above, create a row for every item. 
# This is done by setting index to Item (the column in the original table that contains item names)
# We then are telling pandas we want to create a column for every unique element in the
# original CType column.  And finally, we want the value in the cells to be the value from the USD
# column in the original table.

CType,Bronze,Gold,Silver
Item,,,
Item0,2$,1$,NaN
Item1,NaN,3$,4$


![pivot 1](http://nikgrozev.com/images/blog/Reshaping%20in%20Pandas%20-%20Pivot%20Pivot-Table%20Stack%20and%20Unstack%20explained%20with%20Pictures/pivoting_simple1.png "pivots")

In [4]:
p = metal.pivot(index='Item',columns='CType',values='USD')  # same as above, I'm just remembering the pivot as "p"

# Access the USD cost of Item0 for Gold customers
# first we find the row for Item0/Gold and then we select the USD column and pull out the value
result = metal[((metal.Item == 'Item0') & (metal.CType == 'Gold'))].USD

In [5]:
type(result)

pandas.core.series.Series

In [6]:
# same thing on pivoted table

# here we pull out the row for Item0, grab the Gold column and print the value
p[p.index == 'Item0'].Gold.values

array(['1$'], dtype=object)

In [7]:
# pivot by multiple columns, I want USD and EU prices

In [8]:
metal.pivot(index='Item',columns='CType')

USD                 EU            
CType Bronze Gold Silver Bronze Gold Silver
Item                                       
Item0     2$   1$    NaN     2€   1€    NaN
Item1    NaN   3$     4$    NaN   3€     4€

In [9]:
# Access the USD cost of Item0 for Gold customers
p = metal.pivot(index='Item',columns='CType')
#p.USD
p.USD[p.USD.index == 'Item0'].Gold.values

array(['1$'], dtype=object)

![pivot 2](http://nikgrozev.com/images/blog/Reshaping%20in%20Pandas%20-%20Pivot%20Pivot-Table%20Stack%20and%20Unstack%20explained%20with%20Pictures/pivoting_simple_error.png "pivots")

In [10]:
# What happens if there is a collision? see the problem?  There are two Item0/Golds (see image above)
table = OrderedDict((
    ("Item", ['Item0', 'Item0', 'Item0', 'Item1']),
    ('CType',['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD',  ['1$', '2$', '3$', '4$']),
    ('EU',   ['1€', '2€', '3€', '4€'])
))
metal = DataFrame(table)
p = metal.pivot(index='Item', columns='CType', values='USD')
# will return an error

ValueError: Index contains duplicate entries, cannot reshape

## pivot_tables is your friend
![pivot 1](http://nikgrozev.com/images/blog/Reshaping%20in%20Pandas%20-%20Pivot%20Pivot-Table%20Stack%20and%20Unstack%20explained%20with%20Pictures/pivoting_table_simple1.png "pivots")

In [11]:
# let's create a table to play with
table = OrderedDict((
    ("Item", ['Item0', 'Item0', 'Item0', 'Item1']),
    ('CType',['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD',  [1, 2, 3, 4]),
    ('EU',   [1.1, 2.2, 3.3, 4.4])
))
metal = DataFrame(table)
metal

,Item,CType,USD,EU
0,Item0,Gold,1,1.1
1,Item0,Bronze,2,2.2
2,Item0,Gold,3,3.3
3,Item1,Silver,4,4.4


In [12]:
# pivot_table is  a bit different than pivot... It's the same with the first part
# index, columns, values remain the same as before BUT we added a rule (aggfunc)
# that says: whey you hit a conflict, the way to resolve it is X (in this case
# x is the "mean"... so find the mean of the two numbers)
p = metal.pivot_table(index='Item',columns='CType',values='USD',aggfunc=np.mean)
p

CType,Bronze,Gold,Silver
Item,,,
Item0,2.0,2.0,NaN
Item1,NaN,NaN,4.0


In [14]:
# you could have also resolved the conflict in other ways.  Here we tell it
# to take the "min"
p = metal.pivot_table(index='Item',columns='CType',values='USD',aggfunc=np.min)
p

CType,Bronze,Gold,Silver
Item,,,
Item0,2.0,1.0,NaN
Item1,NaN,NaN,4.0


In [15]:
# pivots are a specific form of stack/unstack, but more on this later

![pivot 1](http://nikgrozev.com/images/blog/Reshaping%20in%20Pandas%20-%20Pivot%20Pivot-Table%20Stack%20and%20Unstack%20explained%20with%20Pictures/stack-unstack1.png "pivots")

## A worked example

In [18]:
# to start let's make a fake dataset: sales of fruit across US states.
# Don't worry about the details here, but basically we'll pretend
# this string is a CSV file and use the standard loading ops
from io import StringIO

TESTDATA=StringIO("""State,Retailer,Fruit,Sales
MI,Walmart,Apple,100
MI,Wholefoods,Apple,150
MI,Kroger,Orange,180
CA,Walmart,Apple,220
CA,Wholefoods,Apple,180
CA,Safeway,Apple,220
CA,Safeway,Orange,110
NY,Walmart,Apple,90
NY,Walmart,Orange,80
NY,Wholefoods,Orange,120
""")

fruit = pd.read_csv(TESTDATA, index_col=None)
fruit

,State,Retailer,Fruit,Sales
0,MI,Walmart,Apple,100
1,MI,Wholefoods,Apple,150
2,MI,Kroger,Orange,180
3,CA,Walmart,Apple,220
4,CA,Wholefoods,Apple,180
5,CA,Safeway,Apple,220
6,CA,Safeway,Orange,110
7,NY,Walmart,Apple,90
8,NY,Walmart,Orange,80
9,NY,Wholefoods,Orange,120


## (a) What is the total sales for each state?
This requires us to group by state, and aggregate sales by taking the sum.

The easiest way of doing this if to use `groupby`

If you execute groupby on the dataframe what you'll get back is an object called DataFrameGroupBy

In [19]:
fruit.groupby('State')

On its own it's a bit useless... it just keeps track of which rows should go into each "pile" (where pile here means a unique group for each state)

If we ask this object to describe itself, you can see what is inside notice that it threw away all the other columns because they were not numerical.  Only "Sales" which is a number, was kept

In [20]:
fruit.groupby('State').describe()

Sales                                                          
      count        mean        std    min    25%    50%    75%    max
State                                                                
CA      4.0  182.500000  51.881275  110.0  162.5  200.0  220.0  220.0
MI      3.0  143.333333  40.414519  100.0  125.0  150.0  165.0  180.0
NY      3.0   96.666667  20.816660   80.0   85.0   90.0  105.0  120.0

Now, if we had another numerical column, let's call it "Sales2," that column would also be kept.  Let's make a fruit2 DataFrame so you can see that:

In [22]:
from io import StringIO

TESTDATA=StringIO("""State,Retailer,Fruit,Sales,Sales2
MI,Walmart,Apple,100,10
MI,Wholefoods,Apple,150,20
MI,Kroger,Orange,180,30
CA,Walmart,Apple,220,20
CA,Wholefoods,Apple,180,40
CA,Safeway,Apple,220,30
CA,Safeway,Orange,110,20
NY,Walmart,Apple,90,40
NY,Walmart,Orange,80,20
NY,Wholefoods,Orange,120,60
""")

fruit2 = pd.read_csv(TESTDATA, index_col=None)
fruit2

,State,Retailer,Fruit,Sales,Sales2
0,MI,Walmart,Apple,100,10
1,MI,Wholefoods,Apple,150,20
2,MI,Kroger,Orange,180,30
3,CA,Walmart,Apple,220,20
4,CA,Wholefoods,Apple,180,40
5,CA,Safeway,Apple,220,30
6,CA,Safeway,Orange,110,20
7,NY,Walmart,Apple,90,40
8,NY,Walmart,Orange,80,20
9,NY,Wholefoods,Orange,120,60


In [23]:
fruit2.groupby("State").describe()

Sales                                                           Sales2  \
      count        mean        std    min    25%    50%    75%    max  count   
State                                                                          
CA      4.0  182.500000  51.881275  110.0  162.5  200.0  220.0  220.0    4.0   
MI      3.0  143.333333  40.414519  100.0  125.0  150.0  165.0  180.0    3.0   
NY      3.0   96.666667  20.816660   80.0   85.0   90.0  105.0  120.0    3.0   

                                                      
       mean        std   min   25%   50%   75%   max  
State                                                 
CA     27.5   9.574271  20.0  20.0  25.0  32.5  40.0  
MI     20.0  10.000000  10.0  15.0  20.0  25.0  30.0  
NY     40.0  20.000000  20.0  30.0  40.0  50.0  60.0

To actually make use of the groupby, we need to tell pandas what to use to measure what's in each group. In other words, I've created a pile for California, a pile for Michigan, and a pile for New York.  I want a number to what's *inside* each pile.  I could ask for the "size" (so how many rows are in each pile), or I could calculate some mathematical function.  For example, if I wanted to know the total sales, I would call "sum."  What happens is pandas goes through every pile, looks at every "row" inside that pile and, for all numerical properties, calculated something.  In this case it's sum... it adds up everything.  So in our original table we had three items for Michigan (Walmart, Kroger, Wholefoods).  This is our Michigan pile.  We then look at numerical properties for Walmart, Kroger, and Wholefoods. In this case Sales.  Because we are using sum() that means add the sales of each.

In [24]:
# What are the total sales for each state?
fruit.groupby('State').sum()  # instead of size()

,Sales
State,
CA,730
MI,430
NY,290


What just happend? A couple of things:
- `groupby()` got first executed on `df`, returning an `DataFrameGroupBy` object. This object itself is useless unless coupled with an aggregation function, such as `sum()`, `mean()`, `max()`, `apply()`. We will talk about `apply()` more in the next week.
- Then, `sum()` got executed on the `DataFrameGroupBy` object, generating the `DataFrame` object you see above. Notice how the table looks different than the original DataFrame `df`? Here are the differences:
  - The `State` column now becomes the index of the DataFrame. The string "State" is the name of the index. Notice how the index name is displayed on a lower level than column names.
  - Since we performed a `groupby` operation by `State`, so only the unique values of `State` are kept as index.
  - Among the other columns, Retailer, Fruit, and Sales, only Sales is kept in the result table. This is because the aggregation function `sum()` only knows how to aggregate numerical values. And only Sales is a numerical column. The other columns are hence dropped.
  
## (b) What is the total sales for each state for each fruit?
This requires us to perform `groupby` on two columns. So, we provide a list of column names to the `groupby` function.

Don't forget that an aggregation function needs to follow the `groupby` function in order to generate results.

In [25]:
# What is the total sales for each state for each fruit?
fruit.groupby(['State','Fruit']).sum()

Sales
State Fruit        
CA    Apple     620
      Orange    110
MI    Apple     250
      Orange    180
NY    Apple      90
      Orange    200

How is this DataFrame different from the previous one?

The biggest different is that this DataFrame has what is called a `MultiIndex` (or hierarchical index), as opposed to a simple index. In this table, the left two "columns" are not columns but actually part of the `MultiIndex`, and the `Sales` is the single real "column" in the DataFrame. (Running out of terminologies here...)

The hierarchical index can be organized in an alternative way if we swapped the order of State and Fruit.

In [26]:
fruit.groupby(['Fruit','State']).sum()

Sales
Fruit  State       
Apple  CA       620
       MI       250
       NY        90
Orange CA       110
       MI       180
       NY       200

## (c) Which state has the maximum total sales?
This question is not asking about the maximum value, but rather which state holds that maximum. There are multiple ways to do it. A principled way is to use `idxmax`.

In [36]:
# Which state has the maximum total sales?
fruitSalesByState = fruit.groupby('State').sum()
print(fruitSalesByState)
max_state = fruitSalesByState['Sales'].idxmax()
print("The state with the maximum sales is: ",max_state)

       Sales
State       
CA       730
MI       430
NY       290
The state with the maximum sales is:  CA




What if I want to display the maximum value alongside the state? Well, we can use that returned label to _select_ the corresponding row from the original DataFrame.

In [37]:
# the deets
fruitSalesByState.loc['CA']

Sales    730
Name: CA, dtype: int64

A less efficient but more intuitive way of doing the same thing:

In [41]:
fruitSalesByState.sort_values('Sales',ascending=False).iloc[0]

Sales    730
Name: CA, dtype: int64

### <font color="magenta">Qx: Which state has the maximum total sales for apples?</font>

In [26]:
# Which state has the maximum total sales for apples?
# give me apple sellers
apples = fruit[fruit.Fruit == 'Apple']
# aggr. by state
applesByState = apples.groupby('State').sum()
# double bracket for dataframe (single for series)
applesByState.loc[[np.argmax(applesByState.Sales)]]

/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


,Sales
State,
CA,620


In the above command, `.loc()` is a function of `DataFrame`. It looks up the index label and returns that row.

The reason I use double square brackets instead of a single pair above is to obtain a `DataFrame` instead of a `Series`, just so that it looks nicer.

## Which state has the biggest difference of sales between apples and oranges?</font>
 (i.e., `abs(#apples-#oranges`))

My solution:
- Step 1: Obtain a table where the rows are states, the columns are fruits, and the cells are total sales (aggregated across retailers).
  - One easy way of doing this is to create pivot table.
  - When creating a pivot table, you will need to always be sharp clear in your mind about three elements:
    - what will be the rows? (This corresponds to the `index` parameter in the `pivot_table` function)
    - what will be the columns?
    - what will be the aggregation functions (hence the meaning of the values)?
- Step 2. Compute absolute difference
- Step 3. Select the state that has the maximum absolute different by applying `np.argmax()`.

In [27]:
# Which state has the biggest difference of sales between apples and oranges?
# (i.e., abs(#apples-#oranges)
fruitPivot = fruit.pivot_table(index='State',columns='Fruit',
                               aggfunc="sum")
fruitPivot

Sales       
Fruit Apple Orange
State             
CA      620    110
MI      250    180
NY       90    200

As you can see, a pivot table is just another `DataFrame`, with slightly more complex indexes and columns.

Let's break this down:
- This `DataFrame` has a simple `Index`. The name of the `Index` is `State`, exactly as we dictated.
- The `columns` of this table is itself a `MultiIndex` (hierarchical index) with two levels.
  - The first level has only one label, "Sales". This level has no name.
  - The second level has two labels, "Apple", and "Orange". This level has a name, "Fruit".

This can be seen by using the commands below.

In [28]:
fruitPivot.columns # give me the heading/multiindex at top

MultiIndex(levels=[['Sales'], ['Apple', 'Orange']],
           labels=[[0, 0], [0, 1]],
           names=[None, 'Fruit'])

In [29]:
np.abs(fruitPivot['Sales']['Apple'] - 
       fruitPivot['Sales']['Orange'])

# to get max
np.argmax(np.abs(fruitPivot['Sales']['Apple'] - 
       fruitPivot['Sales']['Orange']))

/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


'CA'

## (e) Which 2 retailers have the highest total sales in the eastern time zone?
My solution:
- Step 1: Select sale records for Eastern Time Zone states only.
- Step 2: Group by retailers and sum (implying that we will aggregate sales across states)
- Step 3: Sort rows by total sales and select top 2 rows

In [30]:
# Which 2 retailers have the highest total sales in the east coast?
fruitEastern = fruit[fruit.State.isin(['MI','NY'])]
fruitEasternRet = fruitEastern.groupby('Retailer').sum()
fruitEasternRet.sort_values('Sales', ascending=False).head(2)

,Sales
Retailer,
Walmart,270
Wholefoods,270


Note that the parameter in `sort_values`, `ascending=False` means we want the rows to be ordered in descending order.